In [31]:
# load in test waypoints first
import csv
import math
import pandas as pd

waypoint_list = []
with open('/home/teeekaay/team-fusionx/CarND-Capstone/ros/src/waypoint_updater/testwaypoints.csv', 'rb') as Ptfile:
    reader = csv.DictReader(Ptfile, skipinitialspace=True)
    for row in reader:
        waypoint_list.append(row)
#for pt in waypoint_list:
#    print pt['Waypoint'], pt['S']
waypoints = []
    

In [32]:
#JMT code

import numpy as np


class JMT(object):
    def __init__(self, start, end, T):

        """
        Calculates Jerk Minimizing Trajectory for start, end and T.
        start and end include
        [displacement, velocity, acceleration]
        """
        self.start = start
        self.end = end
        self.final_displacement = end[0]
        self.T = T

        a_0, a_1, a_2 = start[0], start[1], start[2] / 2.0
        c_0 = a_0 + a_1 * T + a_2 * T**2
        c_1 = a_1 + 2 * a_2 * T
        c_2 = 2 * a_2

        A = np.array([
                     [T**3,   T**4,    T**5],
                     [3*T**2, 4*T**3,  5*T**4],
                     [6*T,   12*T**2, 20*T**3],
                     ])

        B = np.array([
                     end[0] - c_0,
                     end[1] - c_1,
                     end[2] - c_2
                     ])
        a_3_4_5 = np.linalg.solve(A, B)
        self.coeffs = np.concatenate([np.array([a_0, a_1, a_2]), a_3_4_5])

    # def JMTD_at(self, displacement, coeffs, t0, tmax, deq_wpt_ptr):
    def JMTD_at(self, displacement, t0, tmax):
        # find JMT descriptors at displacement
        s_last = 0.0
        t_found = False
        t_inc = 0.01
        iterations = 0

        for t_cnt in range(int((tmax-t0)/t_inc) + 100):
            iterations += 1
            t = t0 + t_cnt * t_inc
            s = self.get_s_at(t)
            if s > displacement:
                t = t - (1 - (s - displacement) / (s - s_last)) * t_inc
                t_found = True
                break
            # end if
            s_last = s
        # end for
        if t_found is False:
            print("waypoint_updater:JMTD_at Ran out of bounds without "
                          "finding target displacement")
            return None

        s = self.get_s_at(t)
        delta_s = (displacement - s)
        if delta_s > 0.0:
            searchdir = 1.0
        else:
            searchdir = -1.0

        t_smallinc = searchdir * 0.005
        while delta_s * searchdir > 0.0:
            iterations += 1
            t += t_smallinc
            delta_s = displacement - self.get_s_at(t)

#        print("delta_s = {}, t= {}, iterations = {}".format(
#            delta_s, t, iterations))
        if delta_s > 0.1:
            print("waypoint_updater:JMTD_at need to refine algo,"
                          " delta_s is {}".format(delta_s))

        details = JMTDetails(self.get_s_at(t), self.get_v_at(t),
                             self.get_a_at(t), self.get_j_at(t), t)

#        print("waypoint_updater:JMTD_at displacement {} found "
#                       "s,v,a,j,t = {}".format(displacement, details))

        return details

    def get_s_at(self, t):
        return self.coeffs[0] + self.coeffs[1] * t + self.coeffs[2] * t**2 +\
            self.coeffs[3] * t**3 + self.coeffs[4] * t**4 + self.coeffs[5] *\
            t**5

    def get_v_at(self, t):
        return self.coeffs[1] + 2.0 * self.coeffs[2]*t + 3.0 *\
            self.coeffs[3] * t**2 + 4.0 * self.coeffs[4] *\
            t**3 + 5.0 * self.coeffs[5] * t**4

    def get_a_at(self, t):
        return 2.0 * self.coeffs[2] + 6.0 * self.coeffs[3] * t + 12.0 *\
            self.coeffs[4] * t**2 + 20.0 * self.coeffs[5] * t**3

    def get_j_at(self, t):
        return 6.0 * self.coeffs[3] + 24.0 * self.coeffs[4] * t + 60.0 *\
            self.coeffs[5] * t**2


class JMTDetails(object):
    def __init__(self, S, V, A, J, t):
        self.S = S
        self.V = V
        self.A = A
        self.J = J
        self.time = t

    def set_VAJt(self, V, A, J, time):
        self.V = V
        self.A = A
        self.J = J
        self.time = time

    def __repr__(self):
        return "%3.3f, %6.3f, %2.4f, %2.4f, %2.4f" % (self.time, self.S,
                                                      self.V, self.A,
                                                      self.J)


class JMTD_waypoint(object):
    def __init__(self, ptr_id, s):

        self.JMTD = JMTDetails(s, 0.0, 0.0, 0.0, 0.0)
        self.ptr_id = ptr_id
        self.state = None
        self.JMT_ptr = -1  # points to JMT object

    def get_s(self):
        return self.JMTD.S

    def get_a(self):
        return self.JMTD.A

    def get_v(self):
        return self.JMTD.V

    def get_j(self):
        return self.JMTD.J

In [55]:
# functions to build curves


JMT_List = []
waypoints = []
default_accel = 1.3
state = 'stopped'
initial_accel = 0.5
min_moving_velocity = 0.2
handoff_velocity = 1.5
dyn_jmt_time_factor = 1.8

max_accel = 5.0
max_jerk = 5.0
max_velocity = 11.1
default_velocity = 10.7

def get_accel_distance(Vi, Vf, A_avg, Ai=0.0):
    # A_avg is the average acc over the curve more or less
    # but Ai is used if there is an initial acceleration rate to consider
    print("Avg accel = {:3.3f} in get_accel_distance".format(A_avg))
    if Ai == 0.0:  # starts from no acceleration
        if math.fabs(A_avg) < 0.01:
            # avoid divide by 0 - should not happen
            return 50.0  # relatively long distance
        else:
            return math.fabs((Vf**2 - Vi**2)/(2.0 * A_avg))
    elif A_avg/Ai < 0.0:
        # starts with acc in opposite direction
        # when added together will give a smaller value
        # resulting in a longer distance
        return math.fabs((Vf**2 - Vi**2)/(2.0 * (1/3 * Ai + A_avg)))
    else:  # starts with acc in same direction
        return math.fabs((Vf**2 - Vi**2)/(2.0 * (Ai + A_avg)))

def get_accel_time(S, Vi, Vf):
    #does not take into account any initial acceleration
    if (Vi + Vf) < 0.25:
        # avoid divide byfrom geometry_msgs.msg import PoseStamped, TwistStamped
        return (0.0)
    return math.fabs((2.0 * S / (Vi + Vf)))

def get_stopping_distance( ptr_id):
    decel_rate = 1.2
    time_factor = 1.2
    too_short = False
    curpt = waypoints[ptr_id]
    
    a_dist = get_accel_distance(curpt.JMTD.V, 0.0, decel_rate, curpt.JMTD.A)
    T = get_accel_time(a_dist, curpt.JMTD.V, 0.0) * time_factor
    start = [curpt.JMTD.S, curpt.JMTD.V, curpt.JMTD.A]
    end = [curpt.JMTD.S + a_dist, 0.0, 0.0]
    print("Setup JMT to accelerate from v={:3.3f}, a={:3.3f} to v={:3.3f} in dist {:3.3f} m in time {:3.3f} s"
                    .format(curpt.JMTD.V,curpt.JMTD.A, 0.0, a_dist, T))
    jmt = JMT(start, end, T)
    jerk = jmt.get_j_at(0.1)
    print("found jerk of {:3.2f} with a_dist of {:3.2f}".format(jerk, a_dist))
    if jerk < -4.9:
        too_short = True
        dist_diff = 0.5
    else:
        dist_diff = -0.5
    optimized = False
    while optimized == False:
        old_jerk = jerk
        a_dist = a_dist + dist_diff
        end = [curpt.JMTD.S + a_dist, 0.0, 0.0]
        T = get_accel_time(a_dist, curpt.JMTD.V, 0.0) * time_factor
        # print("Setup JMT to accelerate from v={:3.3f}, a={:3.3f} to v={:3.3f} in dist {:3.3f} m in time {:3.3f} s"
        #            .format(curpt.JMTD.V,curpt.JMTD.A, 0.0, a_dist, T))
        jmt = JMT(start, end, T)
        jerk = jmt.get_j_at(0.1)
        # print("found jerk of {:3.2f} with a_dist of {:3.2f}".format(jerk, a_dist))
        if too_short is True:
            if jerk > -4.9:
                final_dist = a_dist
                optimized = True
        else:
            if jerk < -4.9:
                final_dist = a_dist - dist_diff
                jerk = old_jerk
                optimized = True
    print("Setup JMT to decelerate with max_jerk={:3.3f} from v={:3.3f}, a={:3.3f} to v={:3.3f} in dist {:3.3f} m in time {:3.3f} s"
           .format(jerk, curpt.JMTD.V,curpt.JMTD.A, 0.0, final_dist, T))
    return final_dist

def setup_stop_jmt(ptr_id, distance):

    time_factor = 1.2
    curpt = waypoints[ptr_id]
    target_velocity = 0.0

    a_dist = distance
    T = get_accel_time(a_dist, curpt.JMTD.V, target_velocity) * time_factor

    if a_dist < 0.1 or T < 0.1:
        # dummy values to prevent matrix singularity
        # if no velocity change required
        a_dist = 0.1
        T = 0.1

    print("Setup JMT to accelerate from v={:3.3f}, a={:3.3f} to v={:3.3f} in dist {:3.3f} m in time {:3.3f} s"
                    .format(curpt.JMTD.V,curpt.JMTD.A, target_velocity, a_dist, T))

    start = [curpt.JMTD.S, curpt.JMTD.V, curpt.JMTD.A]
    end = [curpt.JMTD.S + a_dist, target_velocity, 0.0]
    jmt = JMT(start, end, T)
    JMT_List.append(jmt)
    jmt_ptr = len(JMT_List)

    return jmt_ptr-1

def setup_jmt( curpt, target_velocity, accel_rate, accel_ratio=1.0,
                time_factor=1.0):
    print("default accel is {}".format(default_accel))

    # accel_ratio set > 1.0 reduces the peak accel rates produced
    # by extending the distance to slow down
            # do this to make acceleration negative on slowdown
    if target_velocity < curpt.JMTD.V:
        accel_ratio = -accel_ratio

    a_dist = get_accel_distance(curpt.JMTD.V, target_velocity,
                                    accel_rate/accel_ratio, curpt.JMTD.A)
    # time_factor values set > 1 stretch out the start or end of the curve
    # where the car is going slowest
    T = get_accel_time(a_dist, curpt.JMTD.V, target_velocity) * time_factor

    if a_dist < 0.1 or T < 0.1:
        # dummy values to prevent matrix singularity
        # if no velocity change required
        a_dist = 0.1
        T = 0.1

    print("Setup JMT to accelerate from v={:3.3f}, a={:3.3f} to v={:3.3f} in dist {:3.3f} m in time {:3.3f} s"
                    .format(curpt.JMTD.V,curpt.JMTD.A, target_velocity, a_dist, T))

    start = [curpt.JMTD.S, curpt.JMTD.V, curpt.JMTD.A]
    end = [curpt.JMTD.S + a_dist, target_velocity, 0.0]
    jmt = JMT(start, end, T)
    JMT_List.append(jmt)
    jmt_ptr = len(JMT_List)

    return jmt_ptr-1
    
def gen_point_in_jmt_curve(pt, jmt_ptr, t):
        # wrapper function to get the JMT details at the displacement
        # of the waypoint and adjust the waypoint values accordingly
    pt.JMT_ptr = jmt_ptr
    JMT_instance = JMT_List[jmt_ptr]
    jmt_pnt = JMT_instance.JMTD_at(pt.get_s(), t, JMT_instance.T * 1.5)
    if jmt_pnt is None:
        print("JMT_at returned None at ptr_id = {}"
                    .format(pt.ptr_id))
        return -1

    pt.JMTD.set_VAJt(jmt_pnt.V, jmt_pnt.A, jmt_pnt.J, jmt_pnt.time)

    #print("velocity set to {} using accel = {} "
    #               "and disp = {} at ptr = {}"
    #               .format(jmt_pnt.V, jmt_pnt.A, pt.get_s(), pt.ptr_id))
    return jmt_pnt.time

def check_point(pt):
    global max_jerk
    global max_velocity
    global max_accel
    # check if JMT values at point exceed desired values
    recalc = False
    if math.fabs(pt.get_a()) > max_accel:
        print("A of {:3.2f} exceeds max value of {:3.2f} "
                      "at ptr = {:3.2f} time={:3.2f}".format
                      (pt.get_a(), max_accel, pt.ptr_id,
                       pt.JMTD.time))
        recalc = True
    if math.fabs(pt.get_j()) > max_jerk:
        print("J of {:3.2f} exceeds max value of {:3.2f} "
                      "at ptr = {:3.2f} time={:3.2f}".format
                      (pt.get_j(), max_jerk, pt.ptr_id,
                       pt.JMTD.time))
        recalc = True
    if math.fabs(pt.get_v()) > max_velocity:
        print("V of {} exceeds max value of {} "
                      "at ptr = {} time={:3.2f}".format
                      (pt.get_v(), max_velocity, pt.ptr_id,
                       pt.JMTD.time))
        recalc = True
    return recalc


def init_acceleration(start_ptr, num_wps, t0=0.0):
    # starts the car moving with a lead foot
    # TODO - rebuild with interpolated waypoints at smaller increments.
    offset = 0
    deltime = t0

    velocity = max(min_moving_velocity, waypoints[start_ptr].get_v())

    waypoints[start_ptr].JMTD.set_VAJt(
                velocity, initial_accel, 0.0, 0.0)
    offset += 1
    while velocity < handoff_velocity and offset < num_wps:
        disp = waypoints[start_ptr + offset].get_s() -\
            waypoints[start_ptr].get_s()
        velocity = min(max(min_moving_velocity,
                           math.sqrt(initial_accel * disp * 2.0)),
                       max_velocity)
        #print("velocity set to {} using accel = {} and "
        #              "disp = {} at ptr = {}"
        #              .format(velocity, initial_accel, disp,
        #                      start_ptr + offset))
        if waypoints[start_ptr + offset].ptr_id > 0:
            deltime += (2.0 *(waypoints[start_ptr + offset].get_s() -\
                waypoints[start_ptr + offset - 1].get_s())/
                (waypoints[start_ptr + offset].get_v() +
                 waypoints[start_ptr + offset - 1].get_v()))
                

        waypoints[start_ptr + offset].JMTD.set_VAJt(
            velocity, initial_accel, 0.0, deltime)
        if state != 'speedup':
            waypoints[start_ptr + offset].JMT_ptr = -1
        offset += 1
    return offset - 1

def generate_speedup(start_ptr, end_ptr, t0=0.0):
    # this is triggered after initiate_aceleration()
    # gets the car moving - that way we can adjust them
    # independently
    global state
    global default_accel
    recalc = False
    curpt = waypoints[start_ptr]


    state = 'speedup'
    acceleration = default_accel
    curpt.JMT_ptr = setup_jmt(curpt, default_velocity, acceleration)
    # accel_ratio = 1.0, time_factor = 1.0)
    jmt_ptr = curpt.JMT_ptr
    JMT_instance = JMT_List[jmt_ptr]

    t = 0.0
    for ptr in range(start_ptr+1, end_ptr):
        mod_ptr = ptr % len(waypoints)
        curpt = waypoints[mod_ptr]

        if curpt.get_s() <= JMT_instance.final_displacement:
            # create the main part of the jmt curve
            t = gen_point_in_jmt_curve(curpt, jmt_ptr, t)
            #fix time for graphs
            curpt.JMTD.time = curpt.JMTD.time + t0
            if check_point(curpt) is True:
                recalc = True
        else:
            # The car has reached the point where it is near to target
            # velocity
            #print("{} beyond S = {} at ptr_id = {}".format
            #               (curpt.get_s(), JMT_instance.final_displacement,
            #                mod_ptr))
            maintain_speed(mod_ptr, 1, waypoints[mod_ptr-1].JMTD.time)
    return recalc

def produce_slowdown(start_ptr, num_wps, accel_ratio, t0=0.0, fastest_stop=False):
    # this generates the deceleration curve based on JMT
    # for the range of waypoints, inserting V = 0.0
    # beyond the stop point
    global state

    recalc = False
    curpt = waypoints[start_ptr]
    target_velocity = 0.0
    if fastest_stop is False:
        jmt_ptr = setup_jmt(curpt, target_velocity, default_accel, accel_ratio,
                                     dyn_jmt_time_factor)
    else:
        stop_dist = get_stopping_distance( start_ptr)
        jmt_ptr = setup_stop_jmt(start_ptr, stop_dist)
        
    curpt.JMT_ptr = jmt_ptr
    state = 'slowdown'
    
    JMT_instance = JMT_List[jmt_ptr]

    t = 0.0
    for ptr in range(start_ptr + 1, start_ptr + num_wps):
        mod_ptr = ptr % len(waypoints)
        curpt = waypoints[mod_ptr]

        if curpt.get_s() <= JMT_instance.final_displacement:
            # create the main part of the jmt curve
            t = gen_point_in_jmt_curve(curpt, jmt_ptr, t)
            if check_point(curpt) is True:
                recalc = True
                        #fix time for graphs
            curpt.JMTD.time = curpt.JMTD.time + t0

        else:
            # the car has reached the point where the JMT
            # curve should reach target velocity of 0.0
            # we smooth the transition
            #print("{} beyond S = {} at ptr_id = {}"
            #               .format(curpt.get_s(),
            #                       JMT_instance.final_displacement,
            #                       mod_ptr))
            set_transition_to_stop(mod_ptr, waypoints[mod_ptr-1].JMTD.time)
    return recalc

def set_transition_to_stop(mod_ptr, t0=0.0):
    # gracefully slow down to stopped at end of jmt decel curve
    deltime = t0
    disp = waypoints[mod_ptr].get_s() -\
        waypoints[mod_ptr-1].get_s()

    decel = waypoints[mod_ptr-1].JMTD.A * 0.5

    velocity = min(max(0.0, waypoints[mod_ptr-1].get_v() -
                       math.sqrt(math.fabs(decel) * disp * 2.0)),
                   max_velocity)
    if velocity < 0.01:
        velocity = 0.0
        decel = 0.0
        if waypoints[mod_ptr-1].get_v()>0:
            deltime += (2.0 *(disp)/
               (velocity + waypoints[mod_ptr - 1].get_v()))    
    else:
        deltime += (2.0 *(disp)/
           (velocity + waypoints[mod_ptr - 1].get_v()))
                
    waypoints[mod_ptr].JMT_ptr = -1
    waypoints[mod_ptr].JMTD.set_VAJt(
        velocity, decel, 0.0, deltime)


def maintain_speed(start_ptr, num_wps, t0=0.0):
    # sets the speed of the car to default velocity for the
    # range of waypoints
    for ptr in range(start_ptr, start_ptr + num_wps):
        mod_ptr = ptr % len(waypoints)
        curpt = waypoints[mod_ptr]
        del_t = t0

        #print("setting to default velocity {:4.3f} at ptr = {}"
        #               .format(default_velocity, mod_ptr))

        velocity = default_velocity

        try:
            del_t += (curpt.get_s() - waypoints[(ptr - 1) %
                      len(waypoints)].get_s()) / velocity
        except NameError:
            del_t = 0.0
        curpt.JMTD.set_VAJt(velocity, 0.0, 0.0, del_t)
        curpt.JMT_ptr = -1

def load_waypoints():
    if len(waypoints) == 0:
        cntr = 0
        s = 0.0
        s_offset = 0
        wpt = None  # just to stop linter complaining

        for row in waypoint_list:
            s = float(row['S'])
            if cntr == 0:
                s_offset = s
            s = s - s_offset
            wpt = JMTD_waypoint(cntr, s)
            waypoints.append(wpt)

            cntr += 1

        print("waypoints_cb {} waypoints loaded, last waypoint "
                      "ptr_id = {} at s= {}".
                      format(len(waypoints), waypoints[cntr-1].
                             ptr_id, waypoints[cntr-1].get_s()))
 
def print_profile():
    print "ptr_id, time, s, v, a, j"
    for wpt in waypoints:
        print("{}, {}, ".format(wpt.ptr_id, wpt.JMTD))

def create_list():
    traj = []
    for wpt in waypoints:
        traj.append((wpt.JMTD.time,wpt.JMTD.S,wpt.JMTD.V,wpt.JMTD.A,wpt.JMTD.J))
    print len(traj)
    return traj

def generate_df():
    traj1 = create_list()
    traj_df = pd.DataFrame(data=traj1,columns=['t','s','v','a','j'])
    return traj_df


In [56]:
import copy
accel_ratio = 1.0
offset = 0.0

def generate_curves(transition = 40, accel_ratio=1.0,time_factor=1.0,default_a=1.5):
    traj_dfs = []
    #global offset
    global waypoints
    global state
    global dyn_jmt_time_factor
    global default_accel
    default_accel = default_a
    dyn_jmt_time_factor = time_factor

    try:
        waypoints = list(copy.deepcopy(stored_waypoints))
    except NameError:
        load_waypoints()
        time_offset=0.0
        state = 'stopped'
        offset = init_acceleration(0, 100, time_offset)
        time_offset = waypoints[offset].JMTD.time
        generate_speedup(offset, offset + 100, time_offset)
        stored_waypoints = list(copy.deepcopy(waypoints)) 
        
    trans_pt = transition
    state = 'speedup'
    # min_stop = get_stopping_distance(trans_pt)
    # print("calculated min stop distance of {:3.2f} m".format(min_stop))
    
    produce_slowdown(trans_pt, 112-trans_pt, accel_ratio, t0=waypoints[trans_pt].JMTD.time, fastest_stop=True)
    traj_dfs.append(copy.deepcopy(generate_df()))


    for df in traj_dfs:
        print "max v = {:3.3f}".format(df['v'].max())
        print "max a = {:3.3f}, min a = {:3.3f}".format(df['a'].max(), df['a'].min())
        print "max j = {:3.3f}, min j = {:3.3f}".format(df['j'].max(), df['j'].min())
        plot_graphs(df)



In [57]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 8]

def plot_graphs(traj_df):
    plt.figure()
    #plt.plot(traj_df.t,traj_df.s,linewidth=.5,color='g')
    plt.plot(traj_df.t,traj_df.v,linewidth=1,color='r',marker='+')
    plt.plot(traj_df.t,traj_df.a,linewidth=1,color='y')
    plt.plot(traj_df.t,traj_df.j,linewidth=1,color='b')
    plt.axis([0, 20, -5, 12.5])
    plt.grid(True)
    plt.title('V, A, J vs time')
    plt.show()
    
    plt.figure()
    #plt.plot(traj_df.t,traj_df.s,linewidth=.5,color='g')
    plt.plot(traj_df.s,traj_df.v,linewidth=1,color='r',marker='+')
    plt.plot(traj_df.s,traj_df.a,linewidth=1,color='y')
    plt.plot(traj_df.s,traj_df.j,linewidth=1,color='b')
    plt.axis([0, 110, -5, 12.5])
    plt.grid(True)
    plt.title('V, A, J vs displacement')
    plt.show()
   




In [58]:
def f(transition_pt, jmt_accel_ratio, jmt_time_ratio, default_accel):
    generate_curves(transition=transition_pt, accel_ratio=jmt_accel_ratio, time_factor=jmt_time_ratio,
                   default_a=default_accel)
    return(transition_pt)

plt.rcParams["figure.figsize"] = [20, 6]
interact(f, transition_pt=widgets.IntSlider(min=2,max=50,step=1,value=20),
        jmt_accel_ratio=widgets.FloatSlider(min=0.5,max=3.0,step=0.1, value=1.0),
        jmt_time_ratio=widgets.FloatSlider(min=0.5,max=3.0,step=0.1, value=1.2),
        default_accel=widgets.FloatSlider(min=0.5,max=3.0,step=0.1, value=1.5));

interactive(children=(IntSlider(value=20, description=u'transition_pt', max=50, min=2), FloatSlider(value=1.0, description=u'jmt_accel_ratio', max=3.0, min=0.5), FloatSlider(value=1.2, description=u'jmt_time_ratio', max=3.0, min=0.5), FloatSlider(value=1.5, description=u'default_accel', max=3.0, min=0.5), Output()), _dom_classes=('widget-interact',))